In [674]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [675]:
from Dependencies.random_circuit_generator_universal import * # Using universal set!
from Dependencies.functions_list import *
import psutil, time, numpy as np, csv
from qiskit_aer import StatevectorSimulator
import qiskit.qasm2, qiskit.qasm3
from mqt import ddsim
import signal, time

In [676]:
# @calculate_execution_time
def get_random_circ_h(n: int,h: int, h_prob: float = None):
    if h_prob == None:
        qc, qr, seed = random_circ_h_const(n, h) # has default value set to 0.125
    else:
        qc, qr, seed = random_circ_h_const(n, h, h_prob)
    return qc, qr, seed
def get_random_circ_d(n: int, d: int):
    qc, qr, seed = random_circ_d_const(n, d)
    return qc, qr, seed
def get_random_circ_g(n: int, g: int):
    qc, qr, seed = random_circ_g_const(n, g)
    return qc, qr, seed

""" # Bell State circuit
# qc = QuantumCircuit(2)
# qc.h([0,1])
# qc.cz(0,1)
# qc.h(1)
# d = qc.depth()
"""

' # Bell State circuit\n# qc = QuantumCircuit(2)\n# qc.h([0,1])\n# qc.cz(0,1)\n# qc.h(1)\n# d = qc.depth()\n'

In [677]:
%time
qc, qr , seed = get_random_circ_h(3,3,0.1)
print(qc.draw(fold = -1))

CPU times: user 6 μs, sys: 0 ns, total: 6 μs
Wall time: 12.4 μs
     ┌───┐                ┌─────┐┌───┐   ┌───┐
q_0: ┤ S ├─■─────────■──■─┤ Sdg ├┤ H ├─■─┤ H ├
     └───┘ │ ┌─────┐ │  │ └─────┘└───┘ │ └───┘
q_1: ──────■─┤ Tdg ├─┼──■──────────────┼──────
     ┌───┐ │ └─────┘ │  │  ┌───┐       │      
q_2: ┤ H ├─■─────────■──■──┤ T ├───────■──────
     └───┘                 └───┘              


In [678]:
# @calculate_execution_time
def get_stvec_poly(qc, n, t, initial_state):
    if n == t :
        return 
    terms, wire_array, max_new_var = create_poly(qc, n)
    assert t == max_new_var, "Value of 't' != 'max_new_var' from the create_poly function."
    # print("terms are: ", terms)
    # print("wires are: ", wire_array)
    ovs = [j[-1] for j in wire_array]
    # print("Output variables are: ", ovs)
    ttb = get_truthtable(terms, n, t, initial_state)
    # print("ttb is: ", ttb)
    return get_statevector(ttb, n, t, ovs, np)
    # counts = {} # : To-Do

def get_stvec_ddsim(qc):
    backend = ddsim.DDSIMProvider().get_backend("statevector_simulator")
    job = backend.run(qc)
    result = job.result()
    return result.get_statevector()

def get_stvec_aer(qc):
    backend = StatevectorSimulator()
    res = backend.run(qc).result()
    return res.get_statevector()


In [679]:
def get_time_poly(qc, n, t, initial_state):
    # Time Calculation for Simulation using polynomial equation
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()
    # When there is no H gate in our circuit
    if n == t : 
        state_vector = np.zeros(1,dtype=complex)
    else:
        state_vector = get_stvec_poly(qc, n, t, initial_state)

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return (state_vector, cpu_time, wall_time)

def get_time_ddsim(qc):
    # Time Calculation for Simulation using DDSIM by MQT
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_ddsim(qc)

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return (state_vector, cpu_time, wall_time)

def get_time_aer(qc):
    # Time Calculation for Simulation using Qiskit's Aer Simulator
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_aer(qc)
    # printing the statevector amplitudes with a threshold

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return (state_vector, cpu_time, wall_time)


In [680]:
def write_results(qc,n,h,h_prob,seed,result):            
    qc_qasm2 = qiskit.qasm2.dumps(qc)
    qc_qasm3 = qiskit.qasm3.dumps(qc)
    qasm2_filename = f'Results/run2/arbitrary_h/circuits/qc_qasm2_n{n}_h{h}_h_prob{h_prob}.qasm2'
    qasm3_filename = f'Results/run2/arbitrary_h/circuits/qc_qasm3_n{n}_h{h}_h_prob{h_prob}.qasm3'
    with open(qasm2_filename, 'w') as file:
        file.write(f"The seed for the random circuit generator is: {seed}\n")
        file.write(qc_qasm2)
    with open(qasm3_filename, 'w') as file:
        file.write(f"The seed for the random circuit generator is: {seed}\n")
        file.write(qc_qasm3)
    with open('Results/run2/arbitrary_h/program_data_h.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(result)

In [683]:
# TODO: Something is mismatching. keep it append mode

def check_stvector(n, stvec_poly, stvec_aer, stvec_ddsim, circuit_filename, stvec_comp_filename, threshold=1e-6):
    stvec_ddsim = np.asanyarray(stvec_ddsim)
    stvec_aer = np.asanyarray(stvec_aer)

    with open(stvec_comp_filename, 'a') as result_file:
        for i in range(len(stvec_aer)):
            reversed_index = int(format(i, f'0{n}b')[::-1], 2)
            # print(f"Reversed bit for i = {i} is: ", reversed_index)
            if np.abs(stvec_aer[i] - stvec_poly[reversed_index]) > threshold:
                result_file.write(f"Mismatch found in circuit: {circuit_filename}\n")
                result_file.write(f"Mismatch in stvec_aer at index {i} and stvec_poly (reversed index {reversed_index})\n")
                result_file.write("\n")  
                break

            if np.abs(stvec_aer[i] - stvec_ddsim[i]) > threshold:
                result_file.write(f"Mismatch found in circuit: {circuit_filename}\n")
                result_file.write(f"Mismatch in stvec_aer at index {i} and stvec_ddsim (reversed index {reversed_index})\n")
                result_file.write("\n")  
                break
    # print("Poly State Vector: ", stvec_poly)
    # print("Aer State Vector: ", stvec_aer)
    # print("ddsim State Vector: ", stvec_ddsim)

### For varying number of H gates

In [684]:
# If I by-mistake run this cell multiple times, past data will be overwritten, so don't use 'w'.
with open('Results/run2/arbitrary_h/program_data_h.csv', 'a', newline='') as file: 
    writer = csv.writer(file)
    writer.writerow(['n', 'h', 'd', 'g', 't', 'h_prob', 'cpu_time_poly', 'wall_time_poly',
                        'cpu_time_ddsim', 'wall_time_ddsim', 'cpu_time_aer', 'wall_time_aer' ])

timeout = 1  # Timeout period in seconds

def timeout_handler(signum, frame):
    raise TimeoutError("Process exceeded time limit")

def execute_with_timeout(func, *args):
    """ Helper function to execute a function with timeout """
    stop_flag = False
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout)
    try:
        result = func(*args)
    except TimeoutError:
        # print(f"Process exceeded {timeout} seconds and was terminated.")
        stop_flag = True
        result = None
    finally:
        signal.alarm(0)  # Disable the alarm
    return result, stop_flag
should_break = False
for h_prob in np.arange(0.05, 0.401, 0.025):  # change 6 to 40.1
    stop_aer = False
    stop_ddsim = False
    for n in range(3, 100):
        stop_poly = False
        for h in range(1, 100):
            # print(f"n = {n}, h = {h}, h_prob = {h_prob}")
            if stop_poly and stop_aer and stop_ddsim:
                break # so that random circ is not created
            qc, qr, seed = get_random_circ_h(n, h, h_prob)
            # print(qc)
            n = qc.width() 
            h = list(instrct.operation.name for _index, instrct in enumerate(qc.data)).count('h')  # Count 'h' gates
            d = qc.depth()  
            g = gate_counts(qc)  
            t = n + h  
            # Initialize the state of the qubits
            initial_state = [0 for _ in range(n)]

            # Timeout for poly computation
            if not stop_poly:
                result, stop_poly = execute_with_timeout(get_time_poly, qc, n, t, initial_state)
                if stop_poly: 
                    print(f"h = {h}, n = {n}, d = {d}, g = {g}")
                    print(f"Poly is stopped after h = {h}, and for above values")
                (stvec_poly, cpu_time_poly, wall_time_poly) = (None,-1,-1) if stop_poly else result  # Set appropriate values

            # Timeout for aer computation
            if not stop_aer:
                result, stop_aer = execute_with_timeout(get_time_aer, qc)
                if stop_aer: 
                    print(f"h = {h}, n = {n}, d = {d}, g = {g}")
                    print(f"Aer is stopped after n = {n}, and for above values")
                (stvec_aer, cpu_time_aer, wall_time_aer) = (None,-1,-1) if stop_aer else result

            # Timeout for ddsim computation
            if not stop_ddsim:
                result, stop_ddsim = execute_with_timeout(get_time_ddsim, qc)
                if stop_ddsim:
                    print(f"h = {h}, n = {n}, d = {d}, g = {g}")
                    print(f"ddsim is stopped after n = {n}, and for above values")
                (stvec_ddsim, cpu_time_ddsim, wall_time_ddsim) = (None,-1,-1) if stop_ddsim else result

            # Store the result for the current configuration
            results = [n, h, d, g, t, h_prob, 
                    round(cpu_time_poly, 6), round(wall_time_poly, 6),
                    round(cpu_time_ddsim, 6), round(wall_time_ddsim, 6),
                    round(cpu_time_aer, 6), round(wall_time_aer, 6)]

            # # If any computation exceeded the timeout, break out of the loop
            # if stop_poly and stop_aer and stop_ddsim:
            #     should_break = True
            #     break

            # Store the circuit in QASM2 and QASM3 format
            write_results(qc,n,h,h_prob,seed,results)
            circuit_filename = f"qc_qasm3_n{n}_h{h}_h_prob{h_prob}.qasm3"
            stvec_comp_filename = f"Results/run2/arbitrary_h/comparing_statevector_results.txt"
            if stvec_aer is not None and stvec_poly is not None and stvec_ddsim is not None:
                check_stvector(n, stvec_poly, stvec_aer, stvec_ddsim, circuit_filename, stvec_comp_filename)
        # if should_break:
            # break

h = 13, n = 3, d = 152, g = 239
Poly is stopped after h = 13, and for above values
h = 13, n = 4, d = 101, g = 201
Poly is stopped after h = 13, and for above values
h = 13, n = 5, d = 106, g = 220
Poly is stopped after h = 13, and for above values
h = 14, n = 6, d = 67, g = 170
Poly is stopped after h = 14, and for above values
h = 13, n = 7, d = 91, g = 253
Poly is stopped after h = 13, and for above values
h = 12, n = 8, d = 129, g = 366
Poly is stopped after h = 12, and for above values
h = 13, n = 9, d = 83, g = 280
Poly is stopped after h = 13, and for above values
h = 13, n = 10, d = 62, g = 227
Poly is stopped after h = 13, and for above values
h = 13, n = 11, d = 65, g = 249
Poly is stopped after h = 13, and for above values
h = 14, n = 12, d = 43, g = 181
Poly is stopped after h = 14, and for above values
h = 98, n = 12, d = 550, g = 2204
ddsim is stopped after n = 12, and for above values
h = 13, n = 13, d = 48, g = 192
Poly is stopped after h = 13, and for above values
h = 

MemoryError: Unable to allocate 512. GiB for an array with shape (34359738368,) and data type complex128

In [ ]:
import signal
import time

# Timeout handler
def timeout_handler(signum, frame):
    raise TimeoutError("Process exceeded time limit")

def your_function():
    # Simulating a long-running task
    time.sleep(10)  # This will take 10 seconds, exceeding the timeout

def run_with_timeout(func, timeout):
    # Set the signal handler to raise TimeoutError after timeout seconds
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout)  # Set the alarm for the timeout period

    try:
        func()  # Run the function
    except TimeoutError:
        print(f"Process exceeded {timeout} seconds and was terminated.")
    finally:
        signal.alarm(0)  # Disable the alarm

# Run with a timeout of 5 seconds
run_with_timeout(your_function, 5)


Process exceeded 5 seconds and was terminated.


#### Rough Work! Excuse me please.

In [ ]:
instructions = [(instruction.operation.name,
                    [qc.find_bit(q).index for q in instruction.qubits]) 
                    for index, instruction in enumerate(qc.data)]
print("instructions of the circuit are: ", instructions)

instructions of the circuit are:  [('sdg', [3]), ('z', [6]), ('cz', [8, 4]), ('ccz', [6, 3, 5]), ('cz', [1, 0]), ('z', [2]), ('z', [1]), ('ccz', [3, 6, 0]), ('sdg', [1]), ('h', [2]), ('tdg', [8]), ('z', [6]), ('sdg', [1]), ('ccz', [7, 5, 6]), ('s', [5]), ('tdg', [0]), ('sdg', [3]), ('tdg', [7]), ('ccz', [3, 1, 2]), ('z', [2]), ('z', [1]), ('sdg', [4]), ('z', [0]), ('t', [0]), ('sdg', [1]), ('z', [3]), ('s', [4]), ('s', [6]), ('z', [5]), ('s', [5]), ('sdg', [6]), ('cz', [5, 8]), ('sdg', [4]), ('s', [0]), ('tdg', [3]), ('cz', [5, 0]), ('t', [0]), ('t', [4]), ('sdg', [8]), ('h', [0]), ('s', [3]), ('s', [6]), ('s', [1]), ('z', [5]), ('sdg', [1]), ('z', [0]), ('cz', [3, 5]), ('z', [5]), ('s', [0]), ('cz', [1, 5]), ('ccz', [2, 6, 4]), ('z', [1]), ('t', [3]), ('z', [0]), ('ccz', [6, 7, 2]), ('z', [2]), ('h', [3]), ('h', [8]), ('t', [8]), ('tdg', [6]), ('t', [4]), ('s', [8]), ('z', [2]), ('z', [6]), ('z', [0]), ('tdg', [5]), ('cz', [1, 8]), ('t', [0]), ('t', [6]), ('h', [5]), ('ccz', [5, 6, 4]

In [ ]:
n = 10
h = 15
qc, qr = get_random_circ(n=n, h=h)
n = qc.width()
h = list(instruction.operation.name for index,
        instruction in enumerate(qc.data)).count('h')
d = qc.depth()
g = gate_counts(qc)
t = n+h
initial_state = [0 for _ in range(n)]
print(gate_counts(qc))
print(qc.count_ops())
# qc.draw(fold=-1)

NameError: name 'get_random_circ' is not defined

In [10]:
stvec_poly, cpu_time_poly, wall_time_poly = get_time_poly(qc, initial_state)
print(cpu_time_poly, wall_time_poly)

Input variables are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Output variables are:  [21, 20, 2, 3, 24, 17, 23, 10, 22, 9]
state vector using boolean equation: 
 [0. 0. 0. ... 0. 0. 0.]
3.421875 3.493514060974121


In [38]:
# # Load circuits from QASM file
# filename = "multiple_circuits.qasm"
# circuits = []

# with open(filename, 'r') as file:
#     qasm_code = file.read()

# # Split QASM code based on markers
# circuit_codes = qasm_code.split("//")

# for code in circuit_codes:
#     code = code.strip()
#     if code:
#         circuit = QuantumCircuit.from_qasm_str(code)
#         circuits.append(circuit)

# Now `circuits` list contains all the reconstructed quantum circuits